<center><img src="https://assets.datacamp.com/production/repositories/6005/datasets/0c64652120b80cfe2762012d7252b439138be223/piggy_bank.jpg" alt="Piggy bank" width="800" height="300"></center>

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `boolean` | Whether the client's credit is in default |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `boolean` | Whether the client has an existing personal loan |

<br>

## campaign

| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `serial` | Campaign ID - primary key |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `boolean` | Outcome of the previous campaign |
| `campaign_outcome` | `boolean` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

<br>

## economics

| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

In [576]:
import pandas as pd
import numpy as np

pd.options.display.max_rows = None
pd.options.display.max_columns = None
# Start coding here...

In [577]:
marketing = pd.read_csv('bank_marketing.csv')

print(marketing.head())

   client_id  age        job  marital    education credit_default housing  \
0          0   56  housemaid  married     basic.4y             no      no   
1          1   57   services  married  high.school        unknown      no   
2          2   37   services  married  high.school             no     yes   
3          3   40     admin.  married     basic.6y             no      no   
4          4   56   services  married  high.school             no      no   

  loan    contact month day_of_week  duration  campaign  pdays  previous  \
0   no  telephone   may         mon       261         1    999         0   
1   no  telephone   may         mon       149         1    999         0   
2   no  telephone   may         mon       226         1    999         0   
3   no  telephone   may         mon       151         1    999         0   
4  yes  telephone   may         mon       307         1    999         0   

      poutcome  emp_var_rate  cons_price_idx  cons_conf_idx  euribor3m  \
0  non

In [578]:
marketing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       41188 non-null  int64  
 1   age             41188 non-null  int64  
 2   job             41188 non-null  object 
 3   marital         41188 non-null  object 
 4   education       41188 non-null  object 
 5   credit_default  41188 non-null  object 
 6   housing         41188 non-null  object 
 7   loan            41188 non-null  object 
 8   contact         41188 non-null  object 
 9   month           41188 non-null  object 
 10  day_of_week     41188 non-null  object 
 11  duration        41188 non-null  int64  
 12  campaign        41188 non-null  int64  
 13  pdays           41188 non-null  int64  
 14  previous        41188 non-null  int64  
 15  poutcome        41188 non-null  object 
 16  emp_var_rate    41188 non-null  float64
 17  cons_price_idx  41188 non-null 

In [579]:
marketing.describe()

,client_id,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
count,41188.00000,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,20593.50000,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,11890.09578,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,0.00000,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,10296.75000,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,20593.50000,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,30890.25000,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,41187.00000,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [580]:
marketing.columns

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration',
       'campaign', 'pdays', 'previous', 'poutcome', 'emp_var_rate',
       'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y'],
      dtype='object')

In [581]:
#Split marketing into client table
client = marketing[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']]

In [582]:
print(client.head())

   client_id  age        job  marital    education credit_default housing loan
0          0   56  housemaid  married     basic.4y             no      no   no
1          1   57   services  married  high.school        unknown      no   no
2          2   37   services  married  high.school             no     yes   no
3          3   40     admin.  married     basic.6y             no      no   no
4          4   56   services  married  high.school             no      no  yes


In [583]:
#rename client_id column to id
client.rename(columns={'client_id': 'id'}, inplace=True)

client.columns

Index(['id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing',
       'loan'],
      dtype='object')

In [584]:
#Split marketing into campaign table
campaign = marketing[['client_id', 'campaign', 'duration', 'pdays', 'previous', 'poutcome', 'y', 'month', 'day_of_week']]

In [585]:
print(campaign.head())

   client_id  campaign  duration  pdays  previous     poutcome   y month  \
0          0         1       261    999         0  nonexistent  no   may   
1          1         1       149    999         0  nonexistent  no   may   
2          2         1       226    999         0  nonexistent  no   may   
3          3         1       151    999         0  nonexistent  no   may   
4          4         1       307    999         0  nonexistent  no   may   

  day_of_week  
0         mon  
1         mon  
2         mon  
3         mon  
4         mon  


In [586]:
#rename columns
campaign.rename(columns={'duration': 'contact_duration', 'previous': 'previous_campaign_contacts', 'y': 'campaign_outcome', 'poutcome': 'previous_outcome', 'campaign': 'number_contacts'}, inplace=True)

campaign.columns

Index(['client_id', 'number_contacts', 'contact_duration', 'pdays',
       'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome',
       'month', 'day_of_week'],
      dtype='object')

In [587]:
#split df into economics table
economics = marketing[['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']]

In [588]:
print(economics.head())

   client_id  emp_var_rate  cons_price_idx  euribor3m  nr_employed
0          0           1.1          93.994      4.857       5191.0
1          1           1.1          93.994      4.857       5191.0
2          2           1.1          93.994      4.857       5191.0
3          3           1.1          93.994      4.857       5191.0
4          4           1.1          93.994      4.857       5191.0


In [589]:
#rename economics columns
economics.rename(columns={'euribor3m': 'euribor_three_months', 'nr_employed': 'number_employed'}, inplace=True)

economics.columns

Index(['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor_three_months',
       'number_employed'],
      dtype='object')

In [590]:
#Clean eduucation column
client['education'].value_counts().sort_index()

basic.4y                4176
basic.6y                2292
basic.9y                6045
high.school             9515
illiterate                18
professional.course     5243
university.degree      12168
unknown                 1731
Name: education, dtype: int64

In [591]:
# replace "." with "_" if not part of a word
client['education'] = client['education'].str.replace(r'\b\.\b', '_')

In [592]:
client['education'].value_counts().sort_index()

basic_4y                4176
basic_6y                2292
basic_9y                6045
high_school             9515
illiterate                18
professional_course     5243
university_degree      12168
unknown                 1731
Name: education, dtype: int64

In [593]:
client['education'] = client['education'].replace('unknown', 'np.nan')

In [594]:
client['education'].value_counts().sort_index()

basic_4y                4176
basic_6y                2292
basic_9y                6045
high_school             9515
illiterate                18
np.nan                  1731
professional_course     5243
university_degree      12168
Name: education, dtype: int64

In [595]:
#To remove periods in job column in client table
client['job'].value_counts().sort_index()

admin.           10422
blue-collar       9254
entrepreneur      1456
housemaid         1060
management        2924
retired           1720
self-employed     1421
services          3969
student            875
technician        6743
unemployed        1014
unknown            330
Name: job, dtype: int64

In [596]:
client['job'] = client['job'].str.replace('.', '')

In [597]:
client['job'].value_counts().sort_index()

admin            10422
blue-collar       9254
entrepreneur      1456
housemaid         1060
management        2924
retired           1720
self-employed     1421
services          3969
student            875
technician        6743
unemployed        1014
unknown            330
Name: job, dtype: int64

In [598]:
campaign['previous_outcome'].value_counts().sort_index()

failure         4252
nonexistent    35563
success         1373
Name: previous_outcome, dtype: int64

In [599]:
#convert values in previous_outcome column in campaign table
campaign['previous_outcome'] = [
    '1' if val == 'success' else 
    '0' if val == 'failure' else 
    np.nan 
    for val in campaign['previous_outcome']
]

In [600]:
campaign['previous_outcome'].value_counts().sort_index()

0    4252
1    1373
Name: previous_outcome, dtype: int64

In [601]:
campaign['campaign_outcome'].value_counts().sort_index()

no     36548
yes     4640
Name: campaign_outcome, dtype: int64

In [602]:
#convert values in campaign_outcome column in campaign table
campaign['campaign_outcome'] = [
    '1' if val == 'yes' else 
    '0' if val == 'no' else 
    np.nan 
    for val in campaign['campaign_outcome']
]

In [603]:
campaign['campaign_outcome'].value_counts().sort_index()

0    36548
1     4640
Name: campaign_outcome, dtype: int64

In [604]:
#Add a column called campaign_id in campaign, where all rows have a value of 1.
campaign['campaign_id'] = 1

In [605]:
campaign['campaign_id'].value_counts().sort_index()

1    41188
Name: campaign_id, dtype: int64

In [606]:
#Create a datetime column called last_contact_date, in the format of "year-month-day"..
#where the year is 2022, and the month and day values are taken from the "month" and "day_of_week" columns...
campaign['month'].value_counts().sort_index()

apr     2632
aug     6178
dec      182
jul     7174
jun     5318
mar      546
may    13769
nov     4101
oct      718
sep      570
Name: month, dtype: int64

In [607]:
campaign['day_of_week'].value_counts().sort_index()

fri    7827
mon    8514
thu    8623
tue    8090
wed    8134
Name: day_of_week, dtype: int64

In [608]:
# Capitalize month and day_of_week columns
campaign["month"] = campaign["month"].str.capitalize()
campaign["day_of_week"] = campaign["day_of_week"].str.capitalize()

# Add year column
campaign["year"] = "2022"

In [609]:
# Add last_contact_date column
campaign["last_contact_date"] = campaign["year"].str.cat([campaign["month"], campaign["day_of_week"]], sep="-")

In [610]:
# Convert to datetime
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], 
                                               format="%Y-%b-%a")

In [611]:
campaign['last_contact_date'].value_counts().sort_index()

2022-03-01      546
2022-04-01     2632
2022-05-01    13769
2022-06-01     5318
2022-07-01     7174
2022-08-01     6178
2022-09-01      570
2022-10-01      718
2022-11-01     4101
2022-12-01      182
Name: last_contact_date, dtype: int64

In [612]:
# Drop unneccessary columns
campaign.drop(columns=["month", "day_of_week", "year"], inplace=True)

In [613]:
campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   number_contacts             41188 non-null  int64         
 2   contact_duration            41188 non-null  int64         
 3   pdays                       41188 non-null  int64         
 4   previous_campaign_contacts  41188 non-null  int64         
 5   previous_outcome            5625 non-null   object        
 6   campaign_outcome            41188 non-null  object        
 7   campaign_id                 41188 non-null  int64         
 8   last_contact_date           41188 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(6), object(2)
memory usage: 2.8+ MB


In [614]:
client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       41188 non-null  object
 5   credit_default  41188 non-null  object
 6   housing         41188 non-null  object
 7   loan            41188 non-null  object
dtypes: int64(2), object(6)
memory usage: 2.5+ MB


In [615]:
economics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             41188 non-null  int64  
 1   emp_var_rate          41188 non-null  float64
 2   cons_price_idx        41188 non-null  float64
 3   euribor_three_months  41188 non-null  float64
 4   number_employed       41188 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 1.6 MB


In [616]:
# save client DataFrame to CSV file without an index
client.to_csv('client.csv', index=False)

In [617]:
# save campaign DataFrame to CSV file without an index
campaign.to_csv('campaign.csv', index=False)

In [618]:
# save economics DataFrame to CSV file without an index
economics.to_csv('economics.csv', index=False)

In [619]:
# create SQL code as a string to create the client table
#create columns for client table

client_table = """CREATE TABLE client (
                id SERIAL PRIMARY KEY, 
                age INTEGER, 
                job TEXT, 
                marital TEXT, 
                education TEXT, 
                credit_default BOOLEAN, 
                housing BOOLEAN, 
                loan BOOLEAN); 
                \copy client from 'client.csv' DELIMITER ',' CSV HEADER"""

print(client_table)

CREATE TABLE client (
                id SERIAL PRIMARY KEY, 
                age INTEGER, 
                job TEXT, 
                marital TEXT, 
                education TEXT, 
                credit_default BOOLEAN, 
                housing BOOLEAN, 
                loan BOOLEAN); 
                \copy client from 'client.csv' DELIMITER ',' CSV HEADER


In [620]:
# create SQL code as a string to create the campaign table
#create columns for campaign table

campaign_table = """CREATE TABLE campaign
(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE    
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""


print(campaign_table)

CREATE TABLE campaign
(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE    
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER



In [621]:
# create SQL code as a string to create the campaign table
#create columns for economics table

economics_table = """CREATE TABLE economics
(
    client_id INTEGER references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""


print(economics_table)

CREATE TABLE economics
(
    client_id INTEGER references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER

